Large amounts of information : components and subsystems, agents, timesteps, frames (for visualisation) 

Original wildfire code

https://medium.com/@tetraktyz/how-to-simulate-wildfires-with-python-6562e2eed266

https://github.com/PedroLFerreira/Cellular-Automaton-for-Wildfires/blob/master/Celular%20Automaton%20for%20Wildfires.ipynb

In [178]:
import numpy as np
import imageio

# Cell States
# 0 = Clear, 1 = Fuel, 2 = Fire

prob = .6                # probability of a cell being fuel
total_time = 300         # simulation time
terrain_size = [300,100] # grid dimensions: 10000 cells

In [179]:
# array : grid at each timestep
states = np.zeros((total_time,*terrain_size)) 

# initialize grid at timestep 0
# create random fuel (0) cells and clear (1) cells
states[0] = np.random.choice([0,1],             # possible cell value
                             size=terrain_size, # size of array
                             p=[1-prob,prob])   # probability of each value


# initiliaze the centre cell at timestep 0 as fire (2)
# set the middle cell on fire!!!
states[0,
       terrain_size[0]//2,
       terrain_size[1]//2
      ] = 2


# at each timestep...
for t in range(1,total_time):
    
    # copy previous grid
    states[t] = states[t-1].copy()      

    # loop through each cell apart from border cells
    for x in range(1,terrain_size[0]-1):
        for y in range(1,terrain_size[1]-1):

            if states[t-1,x,y] == 2: # if cell on fire in last timestep...
                
                # ... if any orthoganolly neighbouring cell contains fuel, 
                # set it on fire
                if states[t-1,x+1,y] == 1: 
                    states[t,x+1,y] = 2
                if states[t-1,x-1,y] == 1:
                    states[t,x-1,y] = 2
                if states[t-1,x,y+1] == 1:
                    states[t,x,y+1] = 2
                if states[t-1,x,y-1] == 1:
                    states[t,x,y-1] = 2
                    
                states[t,x,y] = 0    # ... clear cell
                
                

In [180]:
# a grid to hold a colour representation of each cell 
colored = np.zeros((total_time,*terrain_size,3),
                   dtype=np.uint8)

# Color
for t in range(states.shape[0]): # each timestep
    
    for x in range(states[t].shape[0]): # each row
        
        for y in range(states[t].shape[1]): # each column
            
            value = states[t,x,y].copy() # copy each cell

            if value == 0:
                colored[t,x,y] = [139,69,19] # Clear
            elif value == 1: 
                colored[t,x,y] = [0,255,0]   # Fuel
            elif value == 2: 
                colored[t,x,y] = [255,0,0]   # Burning
            
# Crop
cropped = colored[
    :200,
    1:terrain_size[0]-1,
    1:terrain_size[1]-1
]

imageio.mimsave('./video.gif', cropped)

Modified version without the need for an additional colour plot

In [175]:
import numpy as np
import imageio

# Cell States
# 0 = Clear, 1 = Fuel, 2 = Fire

prob = .6                # probability of a cell being fuel
timesteps = 300         # simulation time
height = 100
width = 100
#terrain_size = [100,100] # grid dimensions: 10000 cells

C = [139,69,19] # Clear
F = [0,255,0] # Fuel
B = [255,0,0]   # Burning

In [177]:
# array : grid at each timestep
#states = np.zeros((total_time,*terrain_size)) 

# # array : grid at each timestep
states = np.zeros((timesteps, height, width, 3), dtype=np.uint8) 

init = np.random.choice([0,1],             # possible cell value
                        size=[height, width], # size of array
                        p=[1-prob,prob])   # probability of each value

init = init.reshape(height, width, 1)

states[0] = np.where(init == 1, F, C)

states[0,height//2, width//2] = B

for t in range(1,timesteps):
    
    states[t] = states[t-1].copy()
    
    for x in range(1,height-1):
        
        for y in range(1,width-1):
            
            if np.array_equal(states[t-1,x,y], np.array(B)):
                
                states[t,x,y] = np.array(C)    # ... clear cell                        

                if np.array_equal(states[t-1,x+1,y], np.array(F)):
                                states[t,x+1,y] = np.array(B)

                if np.array_equal(states[t-1,x-1,y], np.array(F)):
                                states[t,x-1,y] = B    

                if np.array_equal(states[t-1,x,y+1], np.array(F)):
                                states[t,x,y+1] = B    

                if np.array_equal(states[t-1,x,y-1], np.array(F)):
                                states[t,x,y-1] = B   
                                
# Crop
cropped = states[:200, 1:height-1, 1:width-1]

imageio.mimsave('./video_.gif', cropped)

Modified version, further simplified

In [1]:
import numpy as np
import imageio

# Cell States
# 0 = Clear, 1 = Fuel, 2 = Fire

prob = .6               # probability of a cell being fuel
timesteps = 300         # simulation time
height = 100            # grid height
width = 100             # grid width


# RGB colour values to represent cell states
C = np.array([139,69,19]) # Clear
F = np.array([0,255,0]) # Fuel
B = np.array([255,0,0])   # Burning

In [2]:
# array : grid at each timestep
#states = np.zeros((total_time,*terrain_size)) 

# # array : grid at each timestep
states = np.zeros((timesteps, height, width, 3), dtype=np.uint8) 
 
# Create an initial random distribution of clear (0) and fuel (1) cells
init = np.random.choice([0,1],                # possible cell value
                        size=[height, width], # size of array
                        p=[1-prob,prob])      # probability of each value
init = init.reshape(height, width, 1) # 3D array

# use initial random distribution to assign colours
states[0] = np.where(init == 1, F, C) 

# set centre cell on fire
states[0,height//2, width//2] = B

for t in range(1,timesteps):       # At every timestep ...
    states[t] = states[t-1].copy() # ... make a copy of the previous timestep.
    
    for x in range(1,height-1):    # Loop through each cell (x,y value) on the grid        
        for y in range(1,width-1):
            
            if np.array_equal(states[t-1,x,y], B):         # If cell is on fire...
                
                states[t,x,y] = C                          # ... clear cell                        

                if np.array_equal(states[t-1,x+1,y], F):   # ...and set any orthoganol neighbours on fire 
                                states[t,x+1,y] = B

                if np.array_equal(states[t-1,x-1,y], F):
                                states[t,x-1,y] = B    

                if np.array_equal(states[t-1,x,y+1], F):
                                states[t,x,y+1] = B    

                if np.array_equal(states[t-1,x,y-1], F):
                                states[t,x,y-1] = B   
                                
# # Crop
cropped = states[:200, 1:height-1, 1:width-1]   # choose the first 200 timesteps (limit for gif)

imageio.mimsave('./video__.gif', cropped)       # make gif

Using functions and colour values as grid

In [1]:
import numpy as np
import imageio

# Cell States
# 0 = Clear, 1 = Fuel, 2 = Fire

prob = .6               # probability of a cell being fuel
timesteps = 300         # simulation time
height = 100            # grid height
width = 100             # grid width


# RGB colour values to represent cell states
C = np.array([139,69,19]) # Clear
F = np.array([0,255,0]) # Fuel
B = np.array([255,0,0])   # Burning

In [4]:
# array : grid at each timestep
#states = np.zeros((total_time,*terrain_size)) 

def iterate():
    "Cycles through each cell and checks if it should be clear, fuel or burning"
    for t in range(1,timesteps):       # At every timestep ...
        states[t] = states[t-1].copy() # ... make a copy of the previous timestep.

        for x in range(1,height-1):    # Loop through each cell (x,y value) on the grid        
            for y in range(1,width-1):

                if np.array_equal(states[t-1,x,y], B):         # If cell is on fire...

                    states[t,x,y] = C                          # ... clear cell                        

                    if np.array_equal(states[t-1,x+1,y], F):   # ...and set any orthoganol neighbours on fire 
                                    states[t,x+1,y] = B

                    if np.array_equal(states[t-1,x-1,y], F):
                                    states[t,x-1,y] = B    

                    if np.array_equal(states[t-1,x,y+1], F):
                                    states[t,x,y+1] = B    

                    if np.array_equal(states[t-1,x,y-1], F):
                                    states[t,x,y-1] = B   
                                




In [ ]:
# # Crop
def make_gif():
    """MAkes a gif of the simulation"""
    cropped = states[:200, 1:height-1, 1:width-1]   # choose the first 200 timesteps (limit for gif)

    imageio.mimsave('./video__.gif', cropped)       # make gif
    
    



In [ ]:
# # array : grid at each timestep
states = np.zeros((timesteps, height, width, 3), dtype=np.uint8) 
 
# Create an initial random distribution of clear (0) and fuel (1) cells
init = np.random.choice([0,1],                # possible cell value
                        size=[height, width], # size of array
                        p=[1-prob,prob])      # probability of each value
init = init.reshape(height, width, 1) # 3D array

# use initial random distribution to assign colours
states[0] = np.where(init == 1, F, C) 

# set centre cell on fire
states[0,height//2, width//2] = B

iterate()

make_gif()

Using numerical values to simplify simluation. 

In [140]:
import numpy as np
import imageio

prob = .6               # probability of a cell being fuel
timesteps = 300         # simulation time
height = 100            # grid height
width = 100             # grid width


# Cell States
# 0 = Clear, 1 = Fuel, 2 = Fire
C = 0 # Clear
F = 1 # Fuel
B = 2 # Burning

In [141]:
# array : grid at each timestep
#states = np.zeros((total_time,*terrain_size)) 

def iterate():
    "Cycles through each cell and checks if it should be clear, fuel or burning"
    for t in range(1,timesteps):              # At every timestep ...
        states[t] = states[t-1].copy()        # ... make a copy of the previous timestep.
        burn = ma.make_mask(states[t-1]==B)*1 # Make a map of burning cells
        
        if burn.size==1:                      # Quit when fire ends
            break
                
        states[t] = states[t] - burn * 2     # Clear any burning cells from previous timestep                                                  
                            
        states[t,1:-1,1:-1] = states[t,1:-1,1:-1] + (states[t,1:-1,1:-1] *  # Burn any neighbours that are fuel
                           np.sign(
                           burn[ :-2, 1:-1] +
                           burn[1:-1, :-2] +
                           burn[1:-1, 2:] +
                           burn[2:  , 1:-1]
                          )
                                                      )
    

# # array : grid at each timestep
states = np.zeros((timesteps, height, width)) 
 
# Create an initial random distribution of clear (0) and fuel (1) cells
states[0] = np.random.choice([C,F],                # possible cell value
                        size=[height, width], # size of array
                        p=[1-prob,prob])      # probability of each value
#init = init.reshape(height, width, 1) # 3D array

# set centre cell on fire
states[0,height//2, width//2] = B

iterate()

# make_gif()



In [152]:
# a grid to hold a colour representation of each cell 
coloured = np.zeros((timesteps,height, width,3),
                   dtype=np.uint8)


# Color
for t in range(timesteps): # each timestep
    
    coloured[t] = np.where(states[t].reshape(height, width, 1) == 0, np.array([139,69,19]).reshape(1,3) , coloured[t])
    coloured[t] = np.where(states[t].reshape(height, width, 1) == 1, np.array([0,255,0]).reshape(1,3)   , coloured[t])
    coloured[t] = np.where(states[t].reshape(height, width, 1) == 2, np.array([255,0,0]).reshape(1,3)   , coloured[t])
            
# Crop
cropped = colored[
    :200,
    1:height-1,
    1:width-1
]

imageio.mimsave('./video.gif', cropped)

Function to make gif

In [157]:
import numpy as np
import imageio

prob = .6               # probability of a cell being fuel
timesteps = 300         # simulation time
height = 100            # grid height
width = 100             # grid width


# Cell States
# 0 = Clear, 1 = Fuel, 2 = Fire
C = 0 # Clear
F = 1 # Fuel
B = 2 # Burning

In [163]:
# array : grid at each timestep
#states = np.zeros((total_time,*terrain_size)) 

def iterate():
    "Cycles through each cell and checks if it should be clear, fuel or burning"
    for t in range(1,timesteps):                # At every timestep ...
        states[t] = states[t-1].copy()          # ... make a copy of the previous timestep.
        
        burn = ma.make_mask(states[t-1]==B)*1   # Make a map of burning cells
        
        if burn.size==1:                  # If no cells are burning, stop the simulation
            break
                
        states[t] = states[t] - burn * 2  # Extinguish any burning cells from previous timestep
                             
        # Ignite any orthoganol neightbours of burning cells from previous timestep
        states[t,1:-1,1:-1] = states[t,1:-1,1:-1] + (states[t,1:-1,1:-1] * 
                           np.sign(
                           burn[ :-2, 1:-1] +
                           burn[1:-1, :-2] +
                           burn[1:-1, 2:] +
                           burn[2:  , 1:-1]
                          )
                                                      )
        


In [164]:
def make_gif():
    "Crops the simulation to a 200 frames (limit for gif). Saves animation as gif"
    # a grid to hold a colour representation of each cell 
    coloured = np.zeros((timesteps, height, width,3),
                       dtype=np.uint8)


    # Color
    for t in range(timesteps): # each timestep

        coloured[t] = np.where(states[t].reshape(height, width, 1) == 0, np.array([139,69,19]).reshape(1,3) , coloured[t])
        coloured[t] = np.where(states[t].reshape(height, width, 1) == 1, np.array([0,255,0]).reshape(1,3)   , coloured[t])
        coloured[t] = np.where(states[t].reshape(height, width, 1) == 2, np.array([255,0,0]).reshape(1,3)   , coloured[t])

    # Crop
    cropped = coloured[
        :200,
        1:height-1,
        1:width-1
    ]

    imageio.mimsave('./video.gif', cropped)
    
    
    



In [165]:
# # array : grid at each timestep
states = np.zeros((timesteps, height, width)) 
 
# Create an initial random distribution of clear (0) and fuel (1) cells
states[0] = np.random.choice([C,F],                # possible cell value
                        size=[height, width], # size of array
                        p=[1-prob,prob])      # probability of each value
#init = init.reshape(height, width, 1) # 3D array

# set centre cell on fire
states[0,height//2, width//2] = B

iterate()

make_gif()
#make_gif(states)




Another take on the make_gif function, which takes the grid as an input. 

In [160]:
def make_gif(grid):
    "Crops the simulation to a 200 frames (limit for gif). Saves animation as gif"
    # a grid to hold a colour representation of each cell 
    coloured = np.zeros((grid.shape[0], 
                         grid.shape[1], 
                         grid.shape[2], 
                         3),
                       dtype=np.uint8)


    # Color
    for t in range(grid.shape[0]): # each timestep

        coloured[t] = np.where(grid[t].reshape(grid.shape[1], grid.shape[2], 1) == 0, np.array([255,255,255]).reshape(1,3) , coloured[t])
        coloured[t] = np.where(grid[t].reshape(grid.shape[1], grid.shape[2], 1) == 1, np.array([0,0,0]).reshape(1,3)   , coloured[t])
        #coloured[t] = np.where(states[t].reshape(height, width, 1) == 2, np.array([255,0,0]).reshape(1,3)   , coloured[t])
        print(coloured[t]) 
    # Crop
    cropped = coloured[
        :200,
        1:grid.shape[1]-1,
        1:grid.shape[2]-1
    ]

    imageio.mimsave('./game_of_life.gif', cropped)

In [ ]:
def make_gif(grid):
    "Crops the simulation to a 200 frames (limit for gif). Saves animation as gif"
    # a grid to hold a colour representation of each cell 
    coloured = np.zeros((grid.shape[0], 
                         grid.shape[1], 
                         grid.shape[2], 
                         3),
                       dtype=np.uint8)


    # Color
    for t in range(grid.shape[0]): # each timestep

        coloured[t] = np.where(grid[t].reshape(grid.shape[1], grid.shape[2], 1) == 0, np.array([255,255,255]).reshape(1,3) , coloured[t])
        coloured[t] = np.where(grid[t].reshape(grid.shape[1], grid.shape[2], 1) == 1, np.array([0,0,0]).reshape(1,3)   , coloured[t])
        #coloured[t] = np.where(states[t].reshape(height, width, 1) == 2, np.array([255,0,0]).reshape(1,3)   , coloured[t])
        print(coloured[t]) 
    # Crop
    cropped = coloured[
        :200,
        1:grid.shape[1]-1,
        1:grid.shape[2]-1
    ]

    imageio.mimsave('./game_of_life.gif', cropped)

In [12]:
# torroidal surface 
# https://www.geeksforgeeks.org/conways-game-life-python-implementation/
total = int(
    (
     grid[i, (j-1)%N] + 
     grid[i, (j+1)%N] + 
     grid[(i-1)%N, j] + 
     grid[(i+1)%N, j] + 
     grid[(i-1)%N, (j-1)%N] + 
     grid[(i-1)%N, (j+1)%N] + 
     grid[(i+1)%N, (j-1)%N] + 
     grid[(i+1)%N, (j+1)%N]
    ) 
    /255) 
  
            

NameError: name 'grid' is not defined